In [46]:
import cv2
from PIL import Image
import numpy as np
import warnings
from torchvision.utils import *

warnings.simplefilter("ignore")

from optic_disc_segmentation import *
from load_lwnet import *
from utils.get_loaders import *
from algo import *

device = "cuda" if torch.cuda.is_available() else "cpu"
model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8 = load_models()
test_loader = get_test_dataset('tmp/')

img_ww, mask_disc, mask_cup = prediction_eval(model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, test_loader, device)
# save_image(segmented_mask, 'tmp.png')
# cv2.imwrite('tmp.png', img_ww)

# segmented_mask_uint8 = segmented_mask.astype(np.uint8) * 255
# cv2.imwrite('tmp.png', segmented_mask_uint8)
# mask_cup = cv2.imread('N-16-R-Cup-Avg.png', cv2.IMREAD_GRAYSCALE)
# mask_disc = cv2.imread('N-16-R-Disc-Avg.png', cv2.IMREAD_GRAYSCALE)
# isnt = ISNT(mask_cup, mask_disc, 'r')
# print(isnt)
# cv2.circle(mask_cup, (x,y), 0, (0, 255, 255), 3)
# cv2.circle(mask_cup, (g,h), 0, (255, 0, 255), 3)

# cv2.imwrite('tmp_disc.png', mask_disc)
# cv2.imwrite('tmp_cup.png', mask_cup)

INFO: CUDA is available. Using CUDA...
INFO: Using device cuda:0
INFO: Creating dataset with ['tmp'] 
INFO: Creating dataset with 1 examples


In [31]:
from itertools import product

mask_cup_ = mask_cup.copy()
vertical_cup_diameters = []
vertical_cup_len = 0
for i, j in product(range(mask_cup_.shape[0]), range(mask_cup_.shape[1])):    
    if j == mask_cup_.shape[1] - 1:
        vertical_cup_diameters.append(vertical_cup_len)
        vertical_cup_len = 0
    elif mask_cup_[j, i] == 255:
        vertical_cup_len += 1
        # cv2.circle(mask_cup_, (i, j), 0, (150, 0, 0), 1)
        
vertical_cup_diameter = max(vertical_cup_diameters)
print(vertical_cup_diameter)
        
        # cv2.circle(mask_cup_, (i, j), 0, (255, 150, 255), 1)
#         if mask_cup[i, j] != 255:
#             mask_cup[i, j] = 0
#             cv2.circle(mask_cup, (i, j), 0, (255, 255, 255), 1)
        
cv2.imwrite('tmp_cup.png', mask_cup_)


56


True

In [32]:
from itertools import product

mask_cup_ = mask_cup.copy()
horizontal_cup_diameters = []
horizontal_cup_len = 0
for i, j in product(range(mask_cup_.shape[0]), range(mask_cup_.shape[1])):
    if j == mask_cup_.shape[1] - 1:
        horizontal_cup_diameters.append(horizontal_cup_len)
        horizontal_cup_len = 0
    elif mask_cup_[i, j] == 255:
        horizontal_cup_len += 1

horizontal_cup_diameter = max(horizontal_cup_diameters)
print(horizontal_cup_diameter)

cv2.imwrite('tmp_cup.png', mask_cup_)

48


True

In [33]:
x, y = 0, 0
for d in range(len(horizontal_cup_diameters)):
    if horizontal_cup_diameters[d] == horizontal_cup_diameter:
        x = d
        break

for e in range(x, len(horizontal_cup_diameters)):
    if horizontal_cup_diameters[e] != horizontal_cup_diameter:
        y = e
        break

print((x, y))

horizontal_cup_diameter_idx = x + round((y - x) / 2)
print(horizontal_cup_diameter_idx)

(262, 265)
264


In [3]:
x, y = 0, 0
for d in range(len(vertical_cup_diameters)):
    if vertical_cup_diameters[d] == vertical_cup_diameter:
        x = d
        break

for e in range(x, len(vertical_cup_diameters)):
    if vertical_cup_diameters[e] != vertical_cup_diameter:
        y = e
        break

print((x, y))

vertical_cup_diameter_idx = x + round((y - x) / 2)
print(vertical_cup_diameter_idx)

(381, 390)
385


In [42]:
mask_cup_ = mask_cup.copy()
cup_bound_s = 0
# for i, j in product(range(mask_cup.shape[0]), range(mask_cup.shape[1])):  
#     if mask_cup_[j, i] != 255:
#         mask_cup_[j, i] = 150 
for i in range(mask_cup_.shape[0]):
    if mask_cup_[i, vertical_cup_diameter_idx] == 0:
        mask_cup_[i, vertical_cup_diameter_idx] = 150
        cup_bound_s += 1
    elif mask_cup_[i, vertical_cup_diameter_idx] == 255:
        break
#     if mask_cup_[vertical_cup_diameter_idx][i] == 255:
#         # print(i)
#     #     cup_bound_s += 1
#         cv2.circle(mask_cup_, (vertical_cup_diameter_idx, i), 0, (150, 255, 255), -1)
    # elif mask_cup_[vertical_cup_diameter_idx][i] == 255:
    #     break

cv2.imwrite('tmp_cup.png', mask_cup_)
print(cup_bound_s)
np.unique(mask_cup_)

237


array([  0., 150., 255.])

In [40]:
mask_cup_ = mask_cup.copy()

for i in reversed(range(mask_cup_.shape[0])):
    if mask_cup_[i, vertical_cup_diameter_idx] == 0:
        mask_cup_[i, vertical_cup_diameter_idx] = 150
        cup_bound_s += 1
    elif mask_cup_[i, vertical_cup_diameter_idx] == 255:
        break

cv2.imwrite('tmp_cup.png', mask_cup_)

True

In [41]:
mask_cup_ = mask_cup.copy()

for j in reversed(range(mask_cup_.shape[1])):
    if mask_cup_[horizontal_cup_diameter_idx, j] == 0:
        mask_cup_[horizontal_cup_diameter_idx, j] = 150
        cup_bound_s += 1
    elif mask_cup_[horizontal_cup_diameter_idx, j] == 255:
        break

cv2.imwrite('tmp_cup.png', mask_cup_)

True

In [47]:
import numpy as np
import pandas as pd
def ISNT(mask_cup: np.ndarray, mask_disc: np.ndarray, eye: str):
    # ====== Measure vertical and horizontal diameters of the cup and disc ===== #
    horizontal_cup_diameters = []
    horizontal_cup_len = 0
    vertical_cup_diameters = []
    vertical_cup_len = 0
    for i, j in product(range(mask_cup.shape[0]), range(mask_cup.shape[1])):
        if j == mask_cup.shape[1] - 1:
            horizontal_cup_diameters.append(horizontal_cup_len)
            horizontal_cup_len = 0
            vertical_cup_diameters.append(vertical_cup_len)
            vertical_cup_len = 0
        elif mask_cup[i, j] == 255:
            horizontal_cup_len += 1
        elif mask_cup[j, i] == 255:
            vertical_cup_len += 1

    horizontal_cup_diameter = max(horizontal_cup_diameters)            
    vertical_cup_diameter = max(vertical_cup_diameters)
    # print(horizontal_cup_diameter, vertical_cup_diameter)

    horizontal_disc_diameters = []
    horizontal_disc_len = 0
    vertical_disc_diameters = []
    vertical_disc_len = 0
    for i, j in product(range(mask_disc.shape[0]), range(mask_disc.shape[1])):
        if j == mask_disc.shape[1] - 1:
            horizontal_disc_diameters.append(horizontal_disc_len)
            horizontal_disc_len = 0
            vertical_disc_diameters.append(vertical_disc_len)
            vertical_disc_len = 0
        elif mask_disc[i, j] == 255:
            horizontal_disc_len += 1
        elif mask_disc[j, i] == 255:
            vertical_disc_len += 1

    horizontal_disc_diameter = max(horizontal_disc_diameters)            
    vertical_disc_diameter = max(vertical_disc_diameters)
    # print(horizontal_disc_diameter, vertical_disc_diameter)

    # ===== Define coordinates of the cup and disc ===== #
    chx, dhy = 0, 0
    for cx in range(len(horizontal_cup_diameters)):
        if horizontal_cup_diameters[cx] == horizontal_cup_diameter:
            chx = cx
            break
    for cy in range(chx, len(horizontal_cup_diameters)):
        if horizontal_cup_diameters[cy] != horizontal_cup_diameter:
            chy = cy
            break
    # print(chx, chy)
    horizontal_cup_diameter_idx = chx + round((chy - chx) / 2)

    dhx, dhy = 0, 0
    for dx in range(len(horizontal_disc_diameters)):
        if horizontal_disc_diameters[dx] == horizontal_disc_diameter:
            dhx = dx
            break
    for dy in range(dhx, len(horizontal_disc_diameters)):
        if horizontal_disc_diameters[dy] != horizontal_disc_diameter:
            dhy = dy
            break
    # print(dhx, dhy)
    horizontal_disc_diameter_idx = dhx + round((dhy - chx) / 2)

    cvx, cvy = 0, 0
    for cx in range(len(vertical_cup_diameters)):
        if vertical_cup_diameters[cx] == vertical_cup_diameter:
            cvx = cx
            break
    for cy in range(cvx, len(vertical_cup_diameters)):
        if vertical_cup_diameters[cy] != vertical_cup_diameter:
            cvy = cy
            break
    # print(cvx, cvy)
    vertical_cup_diameter_idx = cvx + round((cvy - cvx) / 2)

    dvx, dvy = 0, 0
    for dx in range(len(vertical_disc_diameters)):
        if vertical_disc_diameters[dx] == vertical_disc_diameter:
            dvx = dx
            break
    for dy in range(dvx, len(vertical_disc_diameters)):
        if vertical_disc_diameters[dy] != vertical_disc_diameter:
            dvy = dy
            break
    # print(dvx, dvy)
    vertical_disc_diameter_idx = dvx + round((dvy - dvx) / 2)

    # ===== Calculate ISNT ===== #
    # ISNT_I
    cup_bound_i = 0
    for ci in reversed(range(mask_cup.shape[0])):
        if mask_cup[ci, vertical_cup_diameter_idx] == 0:
            mask_cup[ci, vertical_cup_diameter_idx] = 150
            cup_bound_i += 1
        elif mask_cup[ci, vertical_cup_diameter_idx] == 255:
            break
    # print(cup_bound_i)

    disc_bound_i = 0
    for di in reversed(range(mask_disc.shape[0])):
        if mask_disc[di, vertical_disc_diameter_idx] == 0:
            mask_disc[di, vertical_disc_diameter_idx] = 150
            disc_bound_i += 1
        elif mask_disc[di, vertical_disc_diameter_idx] == 255:
            break
    # print(disc_bound_i)

    isnt_i = cup_bound_i - disc_bound_i

    # ISNT_S
    cup_bound_s = 0
    for ci in range(mask_cup.shape[0]):
        if mask_cup[ci, vertical_cup_diameter_idx] == 0:
            mask_cup[ci, vertical_cup_diameter_idx] = 150
            cup_bound_s += 1
        elif mask_cup[ci, vertical_cup_diameter_idx] == 255:
            break
    # print(cup_bound_s)

    disc_bound_s = 0
    for di in range(mask_disc.shape[0]):
        if mask_disc[di, vertical_disc_diameter_idx] == 0:
            mask_disc[di, vertical_disc_diameter_idx] = 150
            disc_bound_s += 1
        elif mask_disc[di, vertical_disc_diameter_idx] == 255:
            break
    # print(disc_bound_s)

    isnt_s = cup_bound_s - disc_bound_s

    # ISNT_N
    cup_bound_n = 0
    for cj in reversed(range(mask_cup.shape[1])):
        if mask_cup[horizontal_cup_diameter_idx, cj] == 0:
            mask_cup[horizontal_cup_diameter_idx, cj] = 150
            cup_bound_n += 1
        elif mask_cup[horizontal_cup_diameter_idx, cj] == 255:
            break
    # print(cup_bound_n)

    disc_bound_n = 0
    for dj in reversed(range(mask_disc.shape[1])):
        if mask_disc[horizontal_disc_diameter_idx, dj] == 0:
            mask_disc[horizontal_disc_diameter_idx, dj] = 150
            disc_bound_n += 1
        elif mask_disc[horizontal_disc_diameter_idx, dj] == 255:
            break
    # print(disc_bound_n)

    isnt_n = cup_bound_n - disc_bound_n

    # ISNT_T
    cup_bound_t = 0
    for cj in range(mask_cup.shape[1]):
        if mask_cup[horizontal_cup_diameter_idx, cj] == 0:
            mask_cup[horizontal_cup_diameter_idx, cj] = 150
            cup_bound_t += 1
        elif mask_cup[horizontal_cup_diameter_idx, cj] == 255:
            break
    # print(cup_bound_t)

    disc_bound_t = 0
    for dj in range(mask_disc.shape[1]):
        if mask_disc[horizontal_disc_diameter_idx, dj] == 0:
            mask_disc[horizontal_disc_diameter_idx, dj] = 150
            disc_bound_t += 1
        elif mask_disc[horizontal_disc_diameter_idx, dj] == 255:
            break
    # print(disc_bound_t)

    isnt_t = cup_bound_t - disc_bound_t

    if eye == 'r':
        isnt_n, isnt_t = swap(isnt_n, isnt_t)

    pd_isnt = pd.DataFrame({'I': [isnt_i], 'S': [isnt_s], 'N': [isnt_n], 'T': [isnt_t]})

    return pd_isnt.to_dict()





mask_cup_ = mask_cup.copy()
ISNT(mask_cup_)









101 363
219 237


In [ ]:
cup_bound_i = 0

j = list(range(mask_cup.shape[0]))
for q in j:
    if mask_cup[q][374] == 0:
        # cv2.circle(mask_cup, (q, 374), 0, (255, 255, 255), -1)
        cup_bound_i += 1
    elif mask_cup[q][374] == 255:
        break

# cup_bound_i = 0

# j.reverse()
# for q in j:
#     if mask_cup[q][374] == 0:
#         cv2.circle(mask_cup, (q, 374), 0, (255, 255, 255), 2)
#         cup_bound_i += 1
#     elif mask_cup[q][374] == 255:
#         break

cv2.imwrite('tmp_cup.png', mask_cup)


In [ ]:
np.unique(mask_cup)